In [1]:
from urllib.request import urlretrieve
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
#from tqdm import tqdm
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

download('https://s3.amazonaws.com/video.udacity-data.com/topher/2016/December/584f6edd_data/data.zip','data.zip')
#download('https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/signnames.csv','signnames.csv')

print("All the files are downloaded")

def uncompress_features_labels(dir,name):
    if(os.path.isdir(name)):
        print('Data extracted')
    else:
        with ZipFile(dir) as zipf:
            zipf.extractall('data')
uncompress_features_labels('data.zip','data')
#uncompress_features_labels('data1.zip','data1')


def data_Files(mypath):
    onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
    print(onlyfiles)

#data_Files('datalab')
#data_Files('signnames.csv')
#data_Files('data1')
#!ls
print('Done')


All the files are downloaded
Data extracted
Done


In [2]:
import csv
import cv2
import numpy as np

lines=[]
with open('./data/data/driving_log.csv') as csvfile:
    reader= csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images=[]
measurements=[]

for line in lines[1:]:
    source_path=line[0]
    filename=source_path.split('/')[-1]
    current_path= './data/data/IMG/'+ filename
    image= cv2.imread(current_path)
    images.append(image)
    imageAug= cv2.flip(image,1)
    images.append(imageAug)
    measurement= float(line[3])
    measurements.append(measurement)
    measurementAug= measurement * -1
    measurements.append(measurementAug)
    

print('Done')
X_train= np.array(images)

y_train= np.array(measurements)

print('Done')

Done
Done


In [3]:
import numpy as np
print(np.shape(X_train))

(16072, 160, 320, 3)


In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Lambda, Cropping2D

model=Sequential()

#model.add(Flatten(input_shape=(160,320,3)))
#model.add(Dense(1))

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0)),input_shape=(160,320,3)))
model.add(Convolution2D(6,5,5,border_mode='valid'))
model.add(Activation('relu'))
model.add(Convolution2D(12,3,3,border_mode='same'))

model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Activation('relu'))
model.add(Convolution2D(24,5,5, border_mode='same'))

model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Activation('relu'))
model.add(Flatten(input_shape=(7,7,24)))
model.add(Dense(588))

model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(294))
model.add(Dropout(0.5))
model.add(Activation('relu')) 

model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Activation('relu')) 

model.add(Dense(1))


model.compile(loss='mse',optimizer='adam')
model.fit(X_train,y_train,nb_epoch=10,validation_split=0.2,shuffle=True)
model.save('model.h5')
print('Done')

Train on 12857 samples, validate on 3215 samples
Epoch 1/10
12857/12857 [==============================] - 61s - loss: 0.0157 - val_loss: 0.0107
Epoch 2/10
12857/12857 [==============================] - 60s - loss: 0.0106 - val_loss: 0.0104
Epoch 3/10
12857/12857 [==============================] - 60s - loss: 0.0102 - val_loss: 0.0110
Epoch 4/10
12857/12857 [==============================] - 60s - loss: 0.0099 - val_loss: 0.0122
Epoch 5/10
12857/12857 [==============================] - 61s - loss: 0.0095 - val_loss: 0.0101
Epoch 6/10
12857/12857 [==============================] - 60s - loss: 0.0091 - val_loss: 0.0114
Epoch 7/10
12857/12857 [==============================] - 61s - loss: 0.0087 - val_loss: 0.0105
Epoch 8/10
12857/12857 [==============================] - 61s - loss: 0.0082 - val_loss: 0.0106
Epoch 9/10
12857/12857 [==============================] - 61s - loss: 0.0077 - val_loss: 0.0106
Epoch 10/10
12857/12857 [==============================] - 61s - loss: 0.0070 - val_los